In [137]:
import boto3 # The AWS SDK for Python (Boto3) provides a Python API for AWS infrastructure services.
import pandas as pd
from io import StringIO, BytesIO

from datetime import datetime
from datetime import timedelta

## Adapter layer

In [138]:
# read all the files in the bucket
def read_csv_to_df(bucket, key, decoding='utf-8', sep=','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

# write to S3 function
def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True #we just need the confirmation it ran succesfully

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    
    return files

## Application layer

In [139]:
# extract content from files and merge them into one df
def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files if not read_csv_to_df(bucket, obj).empty ], ignore_index=True)
    return df

# execute the aggregations
def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

# write results to the second bucket
def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

# last layer including all previous functions
def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

In [140]:
# application layer but not core

# filter the files on the base of certain date
def return_date_list(arg_date, days_download, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, days_download + 1)]
    return return_date_list

## Main Function

In [141]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2022-01-01'
    src_bucket = 'xetra-1234'
    trg_bucket = 'my-xetra-cz'
    src_format = '%Y-%m-%d'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    days_download=10
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    date_list = return_date_list(arg_date, days_download, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)

In [142]:
# run
main()

## Reading the uploaded file

In [143]:
trg_bucket = 'my-xetra-cz'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20231216_200620.parquet
xetra_daily_report_20231216_201047.parquet
xetra_daily_report_20231217_131737.parquet
xetra_daily_report_20231217_222202.parquet
xetra_daily_report_20231217_222603.parquet


In [144]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20231217_131737.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [145]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...
19142,XS2376095068,2022-01-04,41.19,41.36,41.19,41.36,0,0.77
19143,XS2376095068,2022-01-05,40.99,41.02,40.77,41.02,250,-0.81
19144,XS2376095068,2022-01-06,38.15,37.85,37.85,38.15,7,-7.73
19145,XS2376095068,2022-01-07,36.86,36.71,36.71,37.41,2541,-3.02
